In [ ]:
import json
import os

source_dir = r'F:\prism\source'  
json_dir = r'F:\prism\source_json'  

def does_overlap(box1, box2):
    """Check if two bounding boxes overlap."""
    return not (box1['xmax'] <= box2['xmin'] or box1['xmin'] >= box2['xmax'] or
                box1['ymax'] <= box2['ymin'] or box1['ymin'] >= box2['ymax'])

def check_for_overlap(json_file):
    """Check if any icons in a JSON file overlap."""
    with open(json_file, 'r') as f:
        data = json.load(f)
        if not data:  
            return False
        icons = data.get('outputs', {}).get('object', [])
        for i, icon1 in enumerate(icons):
            for j, icon2 in enumerate(icons):
                if i >= j:  
                    continue
                if does_overlap(icon1.get('bndbox', {}), icon2.get('bndbox', {})):
                    return True
    return False

def delete_overlapping_images():
    """Delete images and JSONs with overlapping icons based on JSON metadata."""
    for json_filename in os.listdir(json_dir):
        json_path = os.path.join(json_dir, json_filename)
        if check_for_overlap(json_path):
            
            image_filename = json_filename.replace('.json', '.jpg')
            image_path = os.path.join(source_dir, image_filename)
            if os.path.exists(image_path):
                os.remove(image_path)
                print(f"Deleted {image_path} due to overlapping icons.")
            else:
                print(f"Image file {image_path} not found.")
            
            os.remove(json_path)
            print(f"Deleted {json_path} due to overlapping icons.")
        else:
            print(f"No overlapping icons found in {json_filename}, or file lacks necessary keys.")

if __name__ == "__main__":
    delete_overlapping_images()
